# Дектораторы

В этом домашнем задании мы напишем собственные дектораторы, которые будут менять системные объекты. Но для начала мы с вами познакомимся с функцией `write`.

In [1]:
import sys

sys.stdout.write('Hello, my friend!')

Hello, my friend!

17

Это метод объектов file-like классов, то есть классов, которые реализуют семантику "Меня можно создать, из меня можно прочитать и в меня можно записать".

Самый главный пример такого объекта -- объект `file`, являющийся результатом вызова фукнции `open()`. Для простоты и универсальности взаимодействия, стандартный ввод и стандартный вывод тоже являются файлами, из которых можно читать и в которые можно писать. 

In [2]:
output = open("./some_test_data.txt", 'w')

In [3]:
output.write('123')

3

In [4]:
output.close()

Как вы могли заметить, функция возвращает число записанных байт. Это важная часть контракта, которую нужно поддержать, если вы хотите как-то подменять эту функцию.

## Задача 1


Для начала, давайте подменим метод `write` у объекта `sys.stdin` на такую функцию, которая перед каждым вызовом оригинальной функции записи данных в `stdout` допечатывает к тексту текущую метку времени.

In [8]:
import sys
from datetime import datetime

def my_write(string_text: str):
    """
    Modifies how print() works by adding the current time 
    to the beginning of the output.
    """
    now = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]: ")
    
    if string_text.startswith("\n"):
        original_write(string_text)
    else:
        original_write(now + string_text)
    
if __name__ == "__main__":
    original_write = sys.stdout.write
    sys.stdout.write = my_write

    print("1, 2, 3")
    
    sys.stdout.write = original_write

[2023-12-16 12:01:23]: 1, 2, 3


Вывод должен был бы быть примерно таким:

```
[2021-12-05 12:00:00]: 1, 2, 3
```

## Задача 2

Упакуйте только что написанный код в декторатор. Весь вывод фукнции должен быть помечен временными метками так, как видно выше.

In [11]:
import sys
from datetime import datetime
from typing import Callable


def timed_output(func: Callable):
    """
    Decorator for the func.
    """
    original_write = sys.stdout.write

    def my_write(string_text: str):
        """
        Modifies how print() works by adding the current time 
        to the beginning of the output.
        """
        now = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]: ")
        
        if string_text.startswith("\n"):
            original_write(string_text)
        else:
            original_write(now + string_text)

    def wrapper(*args, **kwargs):
        """
        Wrapper for the func.
        """
        sys.stdout.write = my_write
        func(*args, **kwargs)
        sys.stdout.write = original_write

    return wrapper


@timed_output
def print_greeting(name: str):
    """
    A function that greets the passed object.
    """
    print(f"Hello, {name}!")


if __name__ == "__main__":
    print_greeting("Nikita")

[2023-12-16 12:01:30]: Hello, Nikita!


Вывод должен быть похож на следующий:

```
[2021-12-05 12:00:00]: Hello, Nikita!
```

## Задача 3

Напишите декторатор, который будет перенаправлять вывод фукнции в файл. 

Подсказка: вы можете заменить объект sys.stdout каким-нибудь другим объектом.

In [13]:
import sys
from typing import Callable


def redirect_output(filepath):
    """
    Decorator that redirects the output of a function to a file.
    """
    class FileWrapper:
        """
        Wrapper class that redirects the output to a file.
        """
        def __init__(self, path: str):
            self.file = None
            self.path = path
            self.original_stdout = sys.stdout

        def __enter__(self):
            """
            Enter method called when entering a 'with' statement.
            """
            self.file = open(self.path, "w")
            sys.stdout = self.file

            return self

        def __exit__(self, ex_type, ex_value, traceback):
            """
            Exit method called when leaving a 'with' statement.
            """
            self.file.close()
            sys.stdout = self.original_stdout

    def decorator(func):
        def wrapper(*args, **kwargs):
            """
            Wrapper function that redirects the function's output to a file.
            """
            with FileWrapper(filepath):
                result = func(*args, **kwargs)
                
            return result

        return wrapper

    return decorator


@redirect_output("./function_output.txt")
def calculate():
    """
    Print matrix of powers of numbers from 1 to 19.
    """
    for power in range(1, 5):
        for num in range(1, 20):
            print(num ** power, end=" ")
        print()


if __name__ == "__main__":
    calculate()